In [21]:
from mido import Message, MidiFile, MidiTrack

In [2]:
import pickle
with open('jsb-chorales-16th.pkl', 'rb') as p:
    data = pickle.load(p, encoding="latin1")

In [24]:
def notes_to_midi(note_array, outpath):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    TICKS_PER_BEAT = 480
    step_time = TICKS_PER_BEAT // 4  # 1 beat per step
    velocity = 64

    previous_notes = set()
    curr_time = 0
    for chord in note_array:
        current_notes = set(chord)

        # Notes to turn off (not in current chord)
        for note in previous_notes - current_notes:
            track.append(Message('note_off', note=note, velocity=velocity, time=curr_time))

        # Notes to turn on (new in current chord)
        for note in current_notes - previous_notes:
            track.append(Message('note_on', note=note, velocity=velocity, time=curr_time))

        previous_notes = current_notes
        curr_time += step_time

    # Turn off all remaining notes at the end
    for note in previous_notes:
        track.append(Message('note_off', note=note, velocity=velocity, time=curr_time))

    mid.save(outpath)

notes_to_midi(data['train'][0], "output.mid")